# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Домашнее задание 5

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

**Баллы за задание:**

* Задача 1 &mdash;  60 баллов





-----

In [ ]:
# Bot check

# HW_ID: phds_hw5
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [ ]:
import time
from tqdm.notebook import tqdm
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as sps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, \
                             DataLoader, RandomSampler, SequentialSampler

from IPython.display import clear_output
from pylab import rcParams

rcParams['figure.figsize'] = 15, 7
%matplotlib inline

sns.set(font_scale=1.3, palette='Set2')

#### **Профиль биология**

### Загрузка данных
В этом задании вы будете работать с данными о распространении COVID-19, их можно скачать [здесь](https://www.kaggle.com/datasets/gpreda/coronavirus-2019ncov). Мы будем предказывать показатель смертности, потому что способы измерения выздоровевших и заболевших разнятся между регионами.

 В данных представлены следующие столбцы:

* `Country/Region` &mdash; страна или регион,

* `Province/State` &mdash; город или населенный пункт,
* `Latitude` &mdash; географическая широта,
* `Longitude` &mdash; географическая долгота,
* `Confirmed` &mdash; кол-во подтвержденных случаев заболевания,
* `Recovered` &mdash; кол-во подтвержденных случаев выздоровления,
* `Deaths` &mdash; кол-во смертей,
* `Date` &mdash; дата.

Выгрузите датасет.

Установите в качестве индекса дату.

Избавьтесь от `None`, если они есть.

Выберите страну и регион, где хотите предсказывать.

Удалите лишние столбцы в выбранных данных.

Постройте графики заболевших, выздоровевших и количества смертей.

#### **Профиль физика**

В этом задании мы будем работать с данными о загрязнении воздуха в Пекине, которые можно скачать [здесь](https://github.com/Afkerian/Beijing-Multi-Site-Air-Quality-Data-Data-Set/blob/main/data/beijing%2Bmulti%2Bsite%2Bair%2Bquality%2Bdata/PRSA2017_Data_20130301-20170228.zip). В этой папке представлены несколько таблиц, данные в них сняты с различных станций. Для дальнейшей работы выберите любую таблицу.

 В данных представлены следующие колонки:

* `year, month, day, hour`  &mdash; год, месяц, день и час измерения

* `PM2.5` концентрация PM2.5 (ug/m^3). `PM2.5` &mdash; описывает мелкие вдыхаемые частицы, диаметр которых обычно составляет 2,5 микрометра и меньше.
* `PM10` концентрация PM10 (ug/m^3). `PM10` &mdash; любые твердые частицы в воздухе диаметром 10 микрометров или меньше, включая дым, пыль, сажу, соли, кислоты и металлы.
* `SO2`: концентрация SO2 (ug/m^3),
* `NO2`: концентрация NO2 (ug/m^3),
* `CO`: концентрация CO (ug/m^3),
* `O3`: концентрация O3 (ug/m^3),
* `TEMP`: температура (degree Celsius),
* `PRES`: давление (hPa=100Pa),
* `DEWP`: температура точки росы (degree Celsius),
* `RAIN`: атмосферные осадки (mm),
* `wd`: направление ветра,
* `WSPM`: скорость ветра (m/s),
* `station`: название станции.

Таргетом в этой задаче будет показатель загрязнения воздуха `PM10`.

Выгрузите данные.

Создайте отдельный столбец в формате даты.

Уберите лишние столбцы.

Очистите от null.

Посмотрите на данные. Постройте временные зависимости для физических параметров.

Попробуйте сделать ещё что-то с данными, потому что модель не сможет определять сразу суточную зависимость, тенденции на месяце и периодичные процессы по сезонам.

Посмотрим, есть ли какая-нибудь суточная периодичность. Постройте небольшой кусок какой-нибудь из зависимостей длиной несколько дней.

Есть ли периодичность?

**Ответ**

Избавимся от суточных изменений, усреднив по дню. Для этого мы не учитывали часы в столбце дней, чтобы было удобнее усреднять.

Установите дату как индекс.

Посмотрим, что получилось. Снова постройте зависимости.

Получилось уже лучше, чем было.

Оставьте те параметры, которые хотите предсказывать. Учтите, что шумные данные ухудшают возможности в предсказании.

## Модели и обучение

### Вспомогательные функции

Напишите класс датасета для данных в виде последовательности.

In [ ]:
class TSDataset(torch.utils.data.Dataset):
    def __init__(self, data, timesteps):
        self.data = <...>
        self.timesteps = <...>

    def __len__(self):
        return <...> # количество подпоследовательностей заданной длины

    def __getitem__(self, index):
        return <...> # верните X и y

Здесь мы создаём функции для обучения, для вывода кривых обучения и для рекурсивного предсказания.

In [ ]:
def plot_learning_curves(history):
    '''
    Функция для вывода лосса и метрики во время обучения.

    :param history: (dict)
        accuracy и loss на обучении и валидации.
    '''
    fig = plt.figure(figsize=(10, 5))
    plt.plot(history['loss'])
    plt.ylabel('Лосс')
    plt.xlabel('Эпоха')
    plt.show()


def train_ts_model(
    model,
    criterion,
    optimizer,
    train_batch_gen,
    num_epochs=50,
    device='cuda'
):
    '''
    Функция для обучения модели и вывода лосса во время обучения.

    :param model: обучаемая модель
    :param criterion: функция потерь
    :param optimizer: метод оптимизации
    :param train_batch_gen: генератор батчей для обучения
    :param num_epochs: количество эпох

    :return: обученная модель
    :return: (dict) loss на обучении ('история' обучения)
    '''
    history = defaultdict(list)

    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        start_time = time.time()

        # Устанавливаем поведение dropout / batch_norm  в обучение
        model.train(True)

        # На каждой 'эпохе' делаем полный проход по данным
        for X_batch, y_batch in train_batch_gen:
            X_batch = X_batch.type('torch.FloatTensor').to(device)
            y_batch = y_batch.type('torch.FloatTensor').to(device)

            logits = model(X_batch)

            loss = criterion(logits.squeeze(), y_batch.squeeze())

            # подсчёт производных
            <...>
            # шаг градиента
            <...>
            # зануление градиентов
            <...>

            train_loss += np.sum(loss.detach().cpu().numpy())

        # Подсчитываем лосс и сохраням в 'историю'
        train_loss /= len(train_batch_gen)
        history['loss'].append(train_loss)

        # clear_output()

        # Печатаем результаты после каждой эпохи
        print('Эпоха {} из {} выполнена за {:.3f} сек.'.format(
            epoch + 1, num_epochs, time.time() - start_time))
        print('  лосс: \t{:.6f}'.format(train_loss))
    plot_learning_curves(history)

    return model, history


def evaluate_ts_model(model, start_seq, test_data, scaler, return_all=False, device='cpu'):
    '''
    Функция для проверки качества модели на обучающем отрезке ряда.

    :param model: обучаемая модель,
    :param start_seq: обучающие данные для первого предсказания,
    :param test_data: тестовые данные.
    :param return_all: возвращать все предсказания или только для 1-го магазина

    :return: результаты предсказания.
    '''
    result = []
    model.train(False)
    input_tensor = torch.FloatTensor(start_seq).to(device).unsqueeze(0)

    with torch.no_grad():
        for i in range(len(test_data)):
            # делаем предсказание, а unsqueeze нужны, чтобы сделать размерность (1, 1, 1) вместо (1)

            logits = model(input_tensor[:, i:, :]).unsqueeze(0)#.unsqueeze(2)

            # присоединяем предсказанное значение к последовательности:
            #                        (1, timestep, 1) -> (1, 1, 1)   по оси 1
            input_tensor = torch.cat((input_tensor,        logits),       1    )

            # обратное преобразование к нормальным числам
            logits = scaler.inverse_transform(logits.cpu().numpy().squeeze(0))

            # результат сохраняем
            result.append(logits.squeeze())

    if return_all:
        return np.array(result)

    return np.array(result)

Создадим класс модели.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_lstm_layers, use_pool=False):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers  = num_lstm_layers
        self.input_size  = input_size

        self.lstm = <...> # LSTM-модель с batch_first=True и dropout=0.2

        # Размерность пространства выхода последнего LSTM-слоя равна hidden_size.
        # Линейный слой нужен, чтобы преобразовать выход LSTM к нужному размеру output_size.
        self.fc = <...> # добавьте линейный слой

        # Пуллинг
        self.pool = <...> # добавьте усредняющий все выходы пуллинг-слой
        self.use_pool = use_pool


    def forward(self, input_seq):

        # инициализируем начальные скрытые состояния
        h_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_size).to(device=input_seq.device)
        c_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_size).to(device=input_seq.device)

        out, (_, _) = self.lstm(input_seq, (h_0, c_0))

        if self.use_pool:
            # берем среднее от векторов для всей последовательности
            out_to_fc = self.pool(out.transpose(1, 2)).squeeze(-1)
        else:
            # берем последний выходной вектор
            out_to_fc = out[:, -1, :]

        return self.fc(out_to_fc)

Напишем функцию для визуализации результатов предсказания.

In [ ]:
def plot_results(y_to_train, y_to_test=None, y_forecast=None):
    """
        Функция для визуализации временного ряда и предсказания.

        Параметры:
            - y_to_train: pd.Series
                Временной ряд, на котором обучалась модель.
            - y_to_test: pd.Series
                Временной ряд, который предсказывает модель.
            - y_forecast: array
                Предсказания модели.
            - plot_conf_int: bool
                Надо ли строить предсказательного интервал.
            - left_bound: array
                Левая граница предсказательного интервала.
            - right_bound: array
                Правая граница предсказательного интервала.
    """
    plt.figure(figsize=(15, 5))
    plt.plot(np.arange(len(y_to_train)), y_to_train, label='train')

    if y_to_test is not None:
        plt.plot(np.arange(len(y_to_train), len(y_to_train) + len(y_to_test)), y_to_test,  label='test')
        if y_forecast is not None:
            plt.plot(np.arange(len(y_to_train), len(y_to_train) + len(y_to_test)), y_forecast, label='prediction')
    plt.legend()
    plt.show()

#### **Профиль биология**


Разделим выборку на тест и трейн.

In [ ]:
test_time = pd.Timestamp('2020-12-01')
train_bio = <...>
test_bio = <...>

Посмотрим на то, что получилось.

In [ ]:
plot_results(train_bio, test_bio)

Преобразуем данные.

Зададим длину подпоследовательности, размер батча, девайс.

In [ ]:
timesteps = <...>
batch_size = <...>

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Преобразуйте к датасету с помощью нашего класса и сделайте генератор батчей.

Инициализируйте модель.

Обучите её.

Создадим предсказания на тесте.

Сравним предсказание с реальностью.

**Вывод**

#### **Профиль физика**


Разделим выборку на тест и трейн.

In [ ]:
test_time_phis = pd.Timestamp('2016-04-01')
train_phis = <...>
test_phis = <...>

Посмотрим на то, что получилось.

In [ ]:
for name in df_phis.columns:
  print(name)
  plot_results(train_phis[name], test_phis[name])

Отнормируем датасет.

Зададим длину последовательности, размер батча и девайс.

In [ ]:
timesteps = <...>
batch_size = <...>

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Инициализируем датасет и создадим dataloader к нему.

Инициализируем модель.

Обучим её.

Получим предсказания на тесте.

Сравним предсказания с истинными значениями.

**Вывод**